Import necessary library. Its recommended to use GPU as runtime

In [1]:
import io
import os
import json
import argparse
import time
import numpy as np
import pandas as pd
import datetime
import tensorflow as tf
import time

from keras import layers
from keras.models import load_model
from pickle import TRUE
from sklearn import utils
from tqdm import tqdm as tqdm
from sklearn import utils
from sklearn import metrics 
from sklearn import preprocessing 
import warnings
warnings.filterwarnings('ignore')

print("GPU Condition", tf.test.is_gpu_available()) # True/False

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Condition True


# **Next Time Prediction Task**

# Preprocessing Part
Upload necessary dataset to your directory. We are using Helpdesk dataset as example.
*   **Make sure the dataset has columns name Case ID, Activity, Complete Timestamp, and Resource (If not, please rename it manually)**

You can download our sample dataset here (https://drive.google.com/drive/folders/1x2en9pUz1ak8IO_6ocE-hpkGylaz1da3?usp=sharing)

In [2]:
#change to your prefered event log
df = pd.read_csv("Helpdesk.csv")

In [3]:
df

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,Variant.1,seriousness,customer,product,responsible_section,seriousness_2,service_level,service_type,support_section,workgroup
0,Case 1,Assign seriousness,Value 1,2012/10/09 14:50:17.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
1,Case 1,Take in charge ticket,Value 1,2012/10/09 14:51:01.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
2,Case 1,Take in charge ticket,Value 2,2012/10/12 15:02:56.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
3,Case 1,Resolve ticket,Value 1,2012/10/25 11:54:26.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
4,Case 1,Closed,Value 3,2012/11/09 12:54:39.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21343,Case 4579,Closed,Value 5,2010/09/02 10:11:00.000,Variant 1,1,Variant 1,Value 1,Value 71,Value 3,Value 1,Value 1,Value 3,Value 1,Value 1,Value 1
21344,Case 4580,Take in charge ticket,Value 6,2012/01/03 09:33:43.000,Variant 18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21345,Case 4580,Wait,Value 6,2012/01/10 15:30:11.000,Variant 18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21346,Case 4580,Resolve ticket,Value 6,2012/01/10 17:07:40.000,Variant 18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1


In [4]:
df = df[['Case ID', 'Activity', 'Complete Timestamp', 'Resource']]
df.columns = ["Case", "Activity", "Timestamp", 'Resource']

In [5]:
df

,Case,Activity,Timestamp,Resource
0,Case 1,Assign seriousness,2012/10/09 14:50:17.000,Value 1
1,Case 1,Take in charge ticket,2012/10/09 14:51:01.000,Value 1
2,Case 1,Take in charge ticket,2012/10/12 15:02:56.000,Value 2
3,Case 1,Resolve ticket,2012/10/25 11:54:26.000,Value 1
4,Case 1,Closed,2012/11/09 12:54:39.000,Value 3
...,...,...,...,...
21343,Case 4579,Closed,2010/09/02 10:11:00.000,Value 5
21344,Case 4580,Take in charge ticket,2012/01/03 09:33:43.000,Value 6
21345,Case 4580,Wait,2012/01/10 15:30:11.000,Value 6
21346,Case 4580,Resolve ticket,2012/01/10 17:07:40.000,Value 6


In [6]:
df["Activity"] = df["Activity"].str.lower()
df["Activity"] = df["Activity"].str.replace(" ", "-")

df["Timestamp"] = df["Timestamp"].str.replace("/", "-")
df["Timestamp"]= pd.to_datetime(df["Timestamp"], dayfirst=True).map(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
df.sort_values(by = ["Timestamp"], inplace = True)

In [7]:
df

,Case,Activity,Timestamp,Resource
16857,Case 3608,assign-seriousness,2010-01-13 08:40:25,Value 2
12863,Case 2748,assign-seriousness,2010-01-13 12:26:04,Value 2
19959,Case 4284,assign-seriousness,2010-01-13 12:30:37,Value 2
7168,Case 1534,assign-seriousness,2010-01-13 13:09:31,Value 2
1864,Case 406,assign-seriousness,2010-01-13 17:25:25,Value 2
...,...,...,...,...
5364,Case 1156,closed,2014-01-03 10:02:38,Value 3
14090,Case 3013,closed,2014-01-03 10:03:58,Value 3
18008,Case 3857,closed,2014-01-03 10:08:18,Value 3
12105,Case 2585,closed,2014-01-03 13:19:48,Value 3


In [8]:
keys = ["[PAD]", "[UNK]"]
activities = list(df["Activity"].unique())
resources = list(df["Resource"].unique())
resources = list(map(str, resources))

keys.extend(activities)
val = range(len(keys))

coded_activity = dict({"x_word_dict":dict(zip(keys, val))})
code_activity_normal = dict({"y_word_dict": dict(zip(activities, range(len(activities))))})
code_resource_normal = dict({"x2_word_dict": dict(zip(resources, range(len(resources))))})

coded_activity.update(code_activity_normal)
coded_activity.update(code_resource_normal)

coded_json = json.dumps(coded_activity)
with open(f"metadata.json", "w") as metadata_file:
    metadata_file.write(coded_json)

In [9]:
train_test_ratio = int(abs(df["Case"].nunique()*0.80))
train_list = df["Case"].unique()[:train_test_ratio]
test_list = df["Case"].unique()[train_test_ratio:]

In [10]:
case_id = "Case"
event_name = "Activity"
event_time = "Timestamp"
case_resource = "Resource"

processed_df = pd.DataFrame(columns = ["case_id", "case_resource", "prefix", "k", "time_passed", "recent_time", "latest_time", "next_time"])
idx = 0
unique_cases = df[case_id].unique()
for _, case in enumerate(tqdm(unique_cases)):
    act = df[df[case_id] == case][event_name].to_list()
    resource = df[df[case_id]== case][case_resource].to_list()
    time = df[df[case_id] == case][event_time].str[:19].to_list()
    time_passed = 0
    latest_diff = datetime.timedelta()
    recent_diff = datetime.timedelta()
    next_time =  datetime.timedelta()
    for i in range(0, len(act)):
        prefix = np.where(i == 0, act[0], " ".join(act[:i+1]))
        resource_act = resource[i]
        if i > 0:
            latest_diff = datetime.datetime.strptime(time[i], "%Y-%m-%d %H:%M:%S") - \
                                datetime.datetime.strptime(time[i-1], "%Y-%m-%d %H:%M:%S")
        if i > 1:
            recent_diff = datetime.datetime.strptime(time[i], "%Y-%m-%d %H:%M:%S")- \
                            datetime.datetime.strptime(time[i-2], "%Y-%m-%d %H:%M:%S")
        latest_time = np.where(i == 0, 0, latest_diff.days)
        recent_time = np.where(i <=1, 0, recent_diff.days)
        time_passed = time_passed + latest_time
        if i+1 < len(time):
            next_time = datetime.datetime.strptime(time[i+1], "%Y-%m-%d %H:%M:%S") - \
                        datetime.datetime.strptime(time[i], "%Y-%m-%d %H:%M:%S")
            next_time_days = str(int(next_time.days))
        else:
            next_time_days = str(1)
        processed_df.at[idx, "case_id"]  = case
        processed_df.at[idx, "case_resource"] = resource_act
        processed_df.at[idx, "prefix"]  =  prefix
        processed_df.at[idx, "k"] = i
        processed_df.at[idx, "time_passed"] = time_passed
        processed_df.at[idx, "recent_time"] = recent_time
        processed_df.at[idx, "latest_time"] =  latest_time
        processed_df.at[idx, "next_time"] = next_time_days
        idx = idx + 1

100%|██████████| 4580/4580 [01:18<00:00, 58.04it/s]


In [11]:
train_df = processed_df[processed_df["case_id"].isin(train_list)]
test_df = processed_df[processed_df["case_id"].isin(test_list)]
train_df.to_csv(f"train.csv", index = False)
test_df.to_csv(f"test.csv", index = False)

# Proposal Architecture (Transformer + Resource Embedding)

In [12]:
class TokenResourceAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, resource_size, vocab_size, embed_dim):
        super(TokenResourceAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.res_emb = layers.Embedding(input_dim=resource_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x, x2):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        x2 = self.res_emb(x2)
        return x + x2 + positions 

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_a = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_b = layers.LayerNormalization(epsilon=1e-6)
        self.dropout_a = layers.Dropout(rate)
        self.dropout_b = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout_a(attn_output, training=training)
        out_a = self.layernorm_a(inputs + attn_output)
        ffn_output = self.ffn(out_a)
        ffn_output = self.dropout_b(ffn_output, training=training)
        return self.layernorm_b(out_a + ffn_output)

def set_trainable(model, transfer_criteria = 'finetune_all', print_details = True):
    # freeze layer 0 to layer l, and unfreeze layer l+1 up until layer L
    #freeze all layers
    if transfer_criteria == 'freeze_all':
        for layer in model.layers:
            layer.trainable = False

    #finetune all layers
    elif transfer_criteria == 'finetune_all':
        for layer in model.layers:
            layer.trainable = True

    #set layers trainable from the specified layer up to the last layer
    else:
        trainable_condition = True
        for layer in model.layers:
            trainable_condition = True
            if layer.name == "transformer_block":
                trainable_condition = False
            layer.trainable = trainable_condition

# Next Time Model

In [13]:
def get_next_time_model(max_case_length, vocab_size, resource_size, base_model_path, transfer_learning, output_dim = 1, 
    embed_dim = 36, num_heads = 4, ff_dim = 64):

    x_inputs = layers.Input(shape=(max_case_length,))
    x2_inputs = layers.Input(shape=(max_case_length))
    time_inputs = layers.Input(shape=(3,)) 
    x = TokenResourceAndPositionEmbedding(max_case_length, resource_size, vocab_size, embed_dim)(x_inputs, x2_inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x_t = layers.Dense(32, activation="relu")(time_inputs)
    x = layers.Concatenate()([x, x_t])
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(output_dim, activation="linear")(x)
    if(transfer_learning):
        transformer = tf.keras.Model(inputs=[x_inputs, x2_inputs, time_inputs], outputs=outputs, name = "next_time_transformer_transfer_learning")
        model_trained = load_model(base_model_path)
        for a in range(8):
            if a == 2 or a == 8:
                continue
            extracted_weights = model_trained.layers[a].get_weights()
            transformer.layers[a].set_weights(extracted_weights)
        
        set_trainable(transformer, transfer_criteria="transformer_block", print_details=True)
    else:
        transformer = tf.keras.Model(inputs=[x_inputs, x2_inputs, time_inputs], outputs=outputs, name = "next_time_transformer")
    transformer.summary()
    return transformer

# Next Time Data Loader

In [14]:
def load_data():
  train_df = pd.read_csv(f"train.csv")
  test_df = pd.read_csv(f"test.csv")

  with open(f"metadata.json", "r") as json_file:
      metadata = json.load(json_file)

  x_word_dict = metadata["x_word_dict"]
  y_word_dict = metadata["y_word_dict"]
  x2_word_dict = metadata["x2_word_dict"]
  max_case_length = get_max_case_length(train_df["prefix"].values)
  vocab_size = len(x_word_dict) 
  resource_size = len(x2_word_dict)
  total_classes = len(y_word_dict)

  return (train_df, test_df, 
      x_word_dict, y_word_dict, x2_word_dict,
      max_case_length, vocab_size, resource_size,
      total_classes)


def prepare_data_next_time(df, x_word_dict, x2_word_dict, max_case_length, time_scaler = None, y_scaler = None, shuffle = True):
  x = df["prefix"].values
  x2 = df["case_resource"].values
  time_x = df[["recent_time", "latest_time", 
          "time_passed"]].values.astype(np.float32)
  y = df["next_time"].values.astype(np.float32)
  if shuffle:
      x, time_x, y, x2 = utils.shuffle(x, time_x, y, x2)

  token_x = list()
  for _x in x:
      token_x.append([x_word_dict[s] for s in _x.split()])

  if time_scaler is None:
      time_scaler = preprocessing.StandardScaler()
      time_x = time_scaler.fit_transform(
          time_x).astype(np.float32)
  else:
      time_x = time_scaler.transform(
          time_x).astype(np.float32)            

  if y_scaler is None:
      y_scaler = preprocessing.StandardScaler()
      y = y_scaler.fit_transform(
          y.reshape(-1, 1)).astype(np.float32)
  else:
      y = y_scaler.transform(
          y.reshape(-1, 1)).astype(np.float32)

  token_x = tf.keras.preprocessing.sequence.pad_sequences(
      token_x, maxlen=max_case_length)
  
  token_x2 = list()
  for _x2 in x2:
      token_x2.append(x2_word_dict[str(_x2)])
  
  token_x = np.array(token_x, dtype=np.float32)
  token_x2_final = np.zeros(token_x.shape)
  for idx, x in enumerate(token_x2):
      token_x2_final[idx, :] = x
  token_x2 = token_x2_final
  time_x = np.array(time_x, dtype=np.float32)
  token_x2 = np.array(token_x2, dtype=np.float32)
  y = np.array(y, dtype=np.float32)

  return token_x, token_x2, time_x, y, time_scaler, y_scaler

def get_max_case_length(train_x):
  train_token_x = list()
  for _x in train_x:
      train_token_x.append(len(_x.split()))
  return max(train_token_x)

# Training

In [15]:
(train_df, test_df, x_word_dict, y_word_dict, x2_word_dict, max_case_length, vocab_size, resource_size, num_output) = load_data()
(train_token_x, train_token_x2, train_time_x, train_token_y, time_scaler, y_scaler) = prepare_data_next_time(train_df, x_word_dict, x2_word_dict, max_case_length)

In [16]:
transformer_model = get_next_time_model(
        max_case_length=max_case_length,
        vocab_size=vocab_size,
        resource_size=resource_size,
        base_model_path=None,
        transfer_learning=False)

Model: "next_time_transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 token_resource_and_position_em  (None, 15, 36)      1908        ['input_1[0][0]',                
 bedding (TokenResourceAndPosit                                   'input_2[0][0]']                
 ionEmbedding)                                                                                    
                                                                              

In [17]:
transformer_model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tf.keras.losses.LogCosh())

In [18]:
transformer_model.fit([train_token_x, train_token_x2, train_time_x], train_token_y, epochs=50, batch_size=512, verbose=2)

Epoch 1/50
34/34 - 5s - loss: 0.3281 - 5s/epoch - 160ms/step
Epoch 2/50
34/34 - 0s - loss: 0.2463 - 310ms/epoch - 9ms/step
Epoch 3/50
34/34 - 0s - loss: 0.1630 - 280ms/epoch - 8ms/step
Epoch 4/50
34/34 - 0s - loss: 0.1303 - 286ms/epoch - 8ms/step
Epoch 5/50
34/34 - 0s - loss: 0.1227 - 278ms/epoch - 8ms/step
Epoch 6/50
34/34 - 0s - loss: 0.1172 - 286ms/epoch - 8ms/step
Epoch 7/50
34/34 - 0s - loss: 0.1101 - 274ms/epoch - 8ms/step
Epoch 8/50
34/34 - 0s - loss: 0.1066 - 268ms/epoch - 8ms/step
Epoch 9/50
34/34 - 0s - loss: 0.1059 - 279ms/epoch - 8ms/step
Epoch 10/50
34/34 - 0s - loss: 0.1061 - 266ms/epoch - 8ms/step
Epoch 11/50
34/34 - 0s - loss: 0.1046 - 289ms/epoch - 9ms/step
Epoch 12/50
34/34 - 0s - loss: 0.1043 - 281ms/epoch - 8ms/step
Epoch 13/50
34/34 - 0s - loss: 0.1027 - 272ms/epoch - 8ms/step
Epoch 14/50
34/34 - 0s - loss: 0.1036 - 270ms/epoch - 8ms/step
Epoch 15/50
34/34 - 0s - loss: 0.1040 - 271ms/epoch - 8ms/step
Epoch 16/50
34/34 - 0s - loss: 0.1029 - 315ms/epoch - 9ms/step
Ep

# Evaluate Model

In [19]:
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, token_x2, test_time_x, test_y, _, _ = prepare_data_next_time(
            test_data_subset, x_word_dict, x2_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, token_x2, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 4.344968
Average MSE across all prefixes: 44.9544
Average RMSE across all prefixes: 6.2524576


# Transfer Learning

Now, lets implement Transfer Learning
*   You can train any dataset and save the model using above code and use that model as base model for transfer learning
*   Or you can download our BPIC2012 Prediction Model as base model in here (https://drive.google.com/drive/folders/1oER0QlOVHy_dgys3YCmqNx9Q6HLNxGT9?usp=sharing). **Rename it to BPIC2012_Base_Model.zip**

**Download whole folder as zip and upload to your directory**

In [27]:
#remove our current transformer model
tf.keras.backend.clear_session()

In [29]:
#your zipped file name
if os.path.exists("BPIC2012"):
  !rm -rf BPIC2012
!unzip BPIC2012_Base_Model.zip

Archive:  BPIC2012_Base_Model.zip
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/remaining_time_ckpt/keras_metadata.pb  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/remaining_time_ckpt/saved_model.pb  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/checkpoint  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/remaining_time_ckpt.data-00000-of-00001  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/next_activity_ckpt.index  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/next_activity_ckpt/saved_model.pb  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/next_activity_ckpt/keras_metadata.pb  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/next_time_ckpt/saved_model.pb  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/next_time_ckpt/variables/variables.index  
  inflating: BPIC2012/transfer_learning_base_BPIC_2012/next_time_ckpt/variables/variables.data-00000-of-00001  
  inflating: BPIC2012/transfer_learning_base_BPIC_

#Training

In [30]:
(train_df, test_df, x_word_dict, y_word_dict, x2_word_dict, max_case_length, vocab_size, resource_size, num_output) = load_data()
(train_token_x, train_token_x2, train_time_x, train_token_y, time_scaler, y_scaler) = prepare_data_next_time(train_df, x_word_dict, x2_word_dict, max_case_length)

In [31]:
transformer_model = get_next_time_model(
        max_case_length=max_case_length,
        vocab_size=vocab_size,
        resource_size=resource_size,
        base_model_path="BPIC2012/transfer_learning_base_BPIC_2012/next_time_ckpt",
        transfer_learning=True)

Model: "next_time_transformer_transfer_learning"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 token_resource_and_position_em  (None, 15, 36)      1908        ['input_1[0][0]',                
 bedding (TokenResourceAndPosit                                   'input_2[0][0]']                
 ionEmbedding)                                                                                    
                                                            

In [32]:
transformer_model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=tf.keras.losses.LogCosh())

In [33]:
transformer_model.fit([train_token_x, train_token_x2, train_time_x], train_token_y, epochs=50, batch_size=512, verbose=2)

Epoch 1/50
34/34 - 2s - loss: 0.3256 - 2s/epoch - 50ms/step
Epoch 2/50
34/34 - 0s - loss: 0.1623 - 450ms/epoch - 13ms/step
Epoch 3/50
34/34 - 0s - loss: 0.1161 - 427ms/epoch - 13ms/step
Epoch 4/50
34/34 - 0s - loss: 0.1116 - 415ms/epoch - 12ms/step
Epoch 5/50
34/34 - 0s - loss: 0.1081 - 410ms/epoch - 12ms/step
Epoch 6/50
34/34 - 0s - loss: 0.1073 - 443ms/epoch - 13ms/step
Epoch 7/50
34/34 - 0s - loss: 0.1062 - 479ms/epoch - 14ms/step
Epoch 8/50
34/34 - 0s - loss: 0.1050 - 426ms/epoch - 13ms/step
Epoch 9/50
34/34 - 0s - loss: 0.1044 - 467ms/epoch - 14ms/step
Epoch 10/50
34/34 - 0s - loss: 0.1033 - 494ms/epoch - 15ms/step
Epoch 11/50
34/34 - 0s - loss: 0.1030 - 481ms/epoch - 14ms/step
Epoch 12/50
34/34 - 0s - loss: 0.1031 - 416ms/epoch - 12ms/step
Epoch 13/50
34/34 - 1s - loss: 0.1022 - 500ms/epoch - 15ms/step
Epoch 14/50
34/34 - 0s - loss: 0.1023 - 432ms/epoch - 13ms/step
Epoch 15/50
34/34 - 0s - loss: 0.1023 - 427ms/epoch - 13ms/step
Epoch 16/50
34/34 - 0s - loss: 0.1014 - 404ms/epoch 

#Evaluate Transfer Learning model

In [34]:
k, maes, mses, rmses = [],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, token_x2, test_time_x, test_y, _, _ = prepare_data_next_time(
            test_data_subset, x_word_dict, x2_word_dict, max_case_length, time_scaler, y_scaler, False)   

        y_pred = transformer_model.predict([test_token_x, token_x2, test_time_x])
        _test_y = y_scaler.inverse_transform(test_y)
        _y_pred = y_scaler.inverse_transform(y_pred)

        k.append(i)
        maes.append(metrics.mean_absolute_error(_test_y, _y_pred))
        mses.append(metrics.mean_squared_error(_test_y, _y_pred))
        rmses.append(np.sqrt(metrics.mean_squared_error(_test_y, _y_pred)))

k.append(i + 1)
maes.append(np.mean(maes))
mses.append(np.mean(mses))
rmses.append(np.mean(rmses))  
print('Average MAE across all prefixes:', np.mean(maes))
print('Average MSE across all prefixes:', np.mean(mses))
print('Average RMSE across all prefixes:', np.mean(rmses))

Average MAE across all prefixes: 3.6048203
Average MSE across all prefixes: 38.19379
Average RMSE across all prefixes: 5.468133
